In [1]:
import os
import subprocess
import re
import exiftool
from PIL import Image
from datetime import datetime
import traceback

自动压缩脚本所在文件夹下所有图片和视频，图片全部压缩为jpg，视频格式不变

顺便清理.aae文件

单线程运行，懒得写多线程

警告：该脚本会自动覆盖原文件！！！文件元数据部分保留(拍摄设备、GPD、时间戳、旋转信息等),一旦覆盖不可撤回

In [2]:
metadataTags = [        # 要复制的元数据类型
    'EXIF:Make',  
    'EXIF:Model',  
    'EXIF:Software',  
    'EXIF:Orientation',
    'EXIF:XResolution',
    'EXIF:YResolution',
    'EXIF:ResolutionUnit',
    'EXIF:ModifyDate',  
    'EXIF:HostComputer',  
    'EXIF:ExposureTime',  
    'EXIF:FNumber',  
    'EXIF:ExposureProgram',
    'EXIF:ISO',
    'EXIF:DateTimeOriginal',  
    'EXIF:CreateDate',  
    'EXIF:OffsetTime',  
    'EXIF:OffsetTimeOriginal',  
    'EXIF:OffsetTimeDigitized',  
    'EXIF:ShutterSpeedValue',  
    'EXIF:ApertureValue',  
    'EXIF:BrightnessValue',  
    'EXIF:ExposureCompensation', 
    'EXIF:MeteringMode',  
    'EXIF:Flash',  
    'EXIF:FocalLength',  
    'EXIF:SubSecTimeOriginal',  
    'EXIF:SubSecTimeDigitized',  
    'EXIF:ExposureMode',
    'EXIF:WhiteBalance',
    'EXIF:ColorSpace',
    # 'EXIF:ExifImageWidth',
    # 'EXIF:ExifImageHeight',
    'EXIF:FocalLengthIn35mmFormat',
    'EXIF:LensInfo',
    'EXIF:LensMake',
    'EXIF:LensModel',  
    'EXIF:CompositeImage',
    'EXIF:GPSLatitudeRef',
    'EXIF:GPSLatitude',
    'EXIF:GPSLongitudeRef',
    'EXIF:GPSLongitude',
    'EXIF:GPSAltitudeRef',
    'EXIF:GPSAltitude',
    'EXIF:GPSTimeStamp',
    'EXIF:GPSSpeedRef',
    'EXIF:GPSSpeed',
    'EXIF:GPSImgDirectionRef',
    'EXIF:GPSImgDirection',
    'EXIF:GPSDestBearingRef',
    'EXIF:GPSDestBearing',
    'EXIF:GPSDateStamp',
    'EXIF:GPSHPositioningError',
    'EXIF:Compression',
    'EXIF:Artist',
    'EXIF:Copyright',
    ]

In [3]:
def extract_metadata(input_file):
    try:
        with exiftool.ExifToolHelper() as et:
            metadata = et.get_tags([input_file], tags=metadataTags)[0]
            params = et.get_tags([input_file], tags=['EXIF:Orientation','File:ImageWidth','File:ImageHeight'])[0]
            return metadata, params['EXIF:Orientation'], params['File:ImageWidth'], params['File:ImageHeight']
        
    except Exception:
        with Image.open(input_file) as img:
            width, height = img.size
            return {}, 1, width, height

In [4]:
def write_metadata(output_file, metadata):
    try:
        with exiftool.ExifToolHelper() as et:
            et.set_tags(
                [output_file],
                tags=metadata,
                params=["-P", "-overwrite_original"]
            )
    except Exception:
        pass

In [5]:
if False:
    input_file = '2019_01_03_20_08_IMG_9379.JPG'

    metadataTags = metadataTags
    command = ['-j'] + [f'-{tag}' for tag in metadataTags] + [input_file]
    with exiftool.ExifTool() as et:
        result = et.execute(*command)

    print(result)

    # 定义输入文件和要设置的元数据标签和值
    input_file = 'test.jpg'
    metadata_to_set = {
        'EXIF:Artist': 'Your Name',
        'EXIF:Copyright': 'Your Copyright',
        "SourceFile": "123.PNG"
    }

    # 构建ExifTool命令
    command = ["-P", "-overwrite_original"]
    for tag, value in metadata_to_set.items():
        command.append(f'-{tag}={value}')
    command.append(input_file)

    # 使用ExifTool的execute方法执行命令
    with exiftool.ExifTool() as et:
        et.execute(*command)

In [6]:
# 这里是反向操作！！用来抵消ffmpeg对旋转的修正
def get_anti_transpose(orientation):
    orientation_map = {
        1: "",          # 正常（不旋转）
        2: "hflip",     # 水平翻转
        3: "transpose=1,transpose=1",  # 旋转 180 度
        4: "vflip",     # 垂直翻转
        5: "transpose=3",  # 顺时针旋转 90 度并垂直翻转
        6: "transpose=2",  # 顺时针旋转 90 度
        7: "transpose=0",  # 逆时针旋转 90 度并垂直翻转
        8: "transpose=1"   # 逆时针旋转 90 度
    }
    return orientation_map.get(orientation, "")

In [7]:
def get_video_duration(input_path):
    # 获取视频总时长
    result = subprocess.run(['ffprobe', '-v', 'error', '-show_entries', 'format=duration', '-of', 'default=noprint_wrappers=1:nokey=1', input_path], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, encoding='utf-8')
    return float(result.stdout)

In [8]:
def run_ffmpeg(ffmpeg_command, input_path, isVedio):
    # 启动ffmpeg进程
    process = subprocess.Popen(ffmpeg_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, encoding='utf-8', text=True)
    
    if isVedio:
        # 用于解析进度的正则表达式
        progress_pattern = re.compile(r"time=(\d+:\d+:\d+\.\d+)")

        total_duration = get_video_duration(input_path)
        while True:
            output = process.stderr.readline()
            if output == '' and process.poll() is not None:
                break
            if output:
                progress_match = progress_pattern.search(output)
                if progress_match:
                    time_str = progress_match.group(1)
                    # 将时间字符串转换为秒数
                    h, m, s = map(float, time_str.split(':'))
                    current_time = h * 3600 + m * 60 + s
                    progress = (current_time / total_duration) * 100
                    print(f"\rffmpeg convert: {input_path}, progress: {progress:.2f}%", end = "")
        print("")
    else:
        print(f"ffmpeg convert: {input_path}")
        
    stdout, stderr = process.communicate()      # 读取输出和错误流
    # 检查退出状态码
    if process.returncode != 0:
        raise Exception("Conversion failed!")

In [9]:
def compress_image(input_path, output_path):
    # 获得元数据
    meta, orientation, original_width, original_height = extract_metadata(input_path)
    # 获取图像的原始分辨率
    transpose = get_anti_transpose(orientation)

    # 分辨率太大则按比例缩小
    if max_size_x<=0 or max_size_y<=0:
        scale = 1
    else:
        scale = max(0.7, min(1, max_size_x/original_width, max_size_y/original_height))     #设个0.7的最小值    
    new_width = scale * original_width
    new_height = scale * original_height

    # 压缩值
    if(new_height * new_width > 3000000):
        compressValue = '5'
    else:
        compressValue = '2'
    
    if transpose != "":
        transpose += ","
    # 构造ffmpeg命令
    ffmpeg_command = [
        'ffmpeg', '-i', input_path,
        '-q:v', compressValue,
        '-vf', f'{transpose}scale={new_width}:{new_height}',
        '-map_metadata', '0',
        '-y',
        output_path
    ]

    # 执行ffmpeg命令
    run_ffmpeg(ffmpeg_command, input_path, False)

    return meta

In [10]:
def compress_video(input_path, output_path):
    # 构造ffmpeg命令
    ffmpeg_command = [
        'ffmpeg', '-i', input_path,
        '-c:v', 'libx265',
        '-pix_fmt', 'yuv420p',
        '-crf', '23',
        '-c:a', 'aac',
        '-map_metadata', '0',
        '-y',
        output_path
    ]

     # 执行ffmpeg命令
    run_ffmpeg(ffmpeg_command, input_path, True)

In [11]:
def delete_file(input_path):
    os.remove(input_path)
    print(f'\nRemove {input_path}')

In [12]:
class process_media:
    def __init__(self, filename):
        self.filename = filename
        self.output_path = None

    def run(self):
        format = self.filename.split('.')[-1].lower()
        if format in to_delete_formats:
            delete_file(self.filename)
        elif format in to_compress_image_formats:
            self.output_path = '_'+self.filename.rsplit('.', 1)[0] + '.jpg'     # 文件名前面加_ ,图片固定压缩成jpg
            metadata = compress_image(self.filename, self.output_path)
            write_metadata(self.output_path, metadata)

        elif format in to_compress_vedio_formats:
            self.output_path = '_'+self.filename        # 文件名前面加_ ,格式不变
            compress_video(self.filename, self.output_path)

    # 直接覆盖旧的媒体文件
    def cover_old_file(self):
        if self.output_path:
            os.remove(self.filename)
            os.rename(self.output_path, self.output_path[1:])
    
    # 出错时清理output
    def try_clear_error_output(self):
        try:
            os.remove(self.output_path)
        except:
            pass

In [13]:
def process_media_in_folder(folder_path):
    # 用绝对路径
    folder_path = os.path.abspath(folder_path)
    for root, _, files in os.walk(folder_path):
        os.chdir(os.path.join(folder_path, root))
        print(f"\n[ directory: {root} ]")
        for filename in files:
            try:
                pm = process_media(filename)
                pm.run()
                pm.cover_old_file()
            except Exception as e:
                pm.try_clear_error_output()
                tb_str = ''.join(traceback.format_exception(None, e, e.__traceback__))
                print(f"\033[91m\n{os.path.join(root, filename)} process error, exception: {e}\nTraceback:\n{tb_str}\033[0m")

In [14]:
# 图片分辨率超过下面设定的，会自动缩小分辨率，但比例不变  //////////  -1表示尺寸不变
max_size_x = 3024
max_size_y = 3024
# max_size_x = -1
# max_size_y = -1
#文件路径，会包括其子文件夹
path = "E:\照片\脚本_压缩全部媒体文件"

In [15]:
#将压缩的图片的格式
to_compress_image_formats = ['jpg','png','jpeg']
#将压缩的视频格式
to_compress_vedio_formats = ['mov','mp4']
#要删除的格式
to_delete_formats = ['aae']

In [16]:
process_media_in_folder(path)
print('\ndone...')


[ directory: E:\照片\脚本_压缩全部媒体文件 ]

[ directory: E:\照片\脚本_压缩全部媒体文件\py_compress_media ]

[ directory: E:\照片\脚本_压缩全部媒体文件\py_compress_media\.git ]

[ directory: E:\照片\脚本_压缩全部媒体文件\py_compress_media\.git\hooks ]

[ directory: E:\照片\脚本_压缩全部媒体文件\py_compress_media\.git\info ]

[ directory: E:\照片\脚本_压缩全部媒体文件\py_compress_media\.git\logs ]

[ directory: E:\照片\脚本_压缩全部媒体文件\py_compress_media\.git\logs\refs ]

[ directory: E:\照片\脚本_压缩全部媒体文件\py_compress_media\.git\logs\refs\heads ]

[ directory: E:\照片\脚本_压缩全部媒体文件\py_compress_media\.git\logs\refs\remotes ]

[ directory: E:\照片\脚本_压缩全部媒体文件\py_compress_media\.git\logs\refs\remotes\origin ]

[ directory: E:\照片\脚本_压缩全部媒体文件\py_compress_media\.git\objects ]

[ directory: E:\照片\脚本_压缩全部媒体文件\py_compress_media\.git\objects\01 ]

[ directory: E:\照片\脚本_压缩全部媒体文件\py_compress_media\.git\objects\04 ]

[ directory: E:\照片\脚本_压缩全部媒体文件\py_compress_media\.git\objects\11 ]

[ directory: E:\照片\脚本_压缩全部媒体文件\py_compress_media\.git\objects\1d ]

[ directory: E:\照片\脚本_压缩全部媒体文件\py_c

In [17]:

    
# input_file = 'output.mp4'
# input_file = 'IMG_8908.jpeg'
# output_file = 'IMG_8908.JPG'

# metadata = extract_metadata(input_file)

# if metadata:
#     write_metadata(output_file, metadata)
#     print(f"Metadata written to {output_file}")
# else:
#     print("No metadata found. Output file not created.")
